In [1]:
import numpy as np
import pandas as pd
from os import listdir
import matplotlib.pyplot as plt
import itertools as it
from statsmodels.sandbox.stats.multicomp import multipletests
import sklearn.preprocessing as pp
import statsmodels.api as sm
#import nltk
import scipy.stats as st
import statsmodels.formula.api as smf
import seaborn as sns
import Helper as hp

#Load Data
data = pd.read_csv("CHI_2019_FULL.csv")

#Set Labels 
emotion_label = ['Anger', 'Boredom', 'Disgust', 'Fear', 'Happiness', 'Emo_Neutral', 'Sadness']
affect_label = ['Aggressiv', 'Cheerful', 'Intoxicated', 'Nervous', 'Aff_Neutral', 'Tired']
loi_label = ['Disinterest', 'Normal', 'High Interest']

#Get specific data and save it into new data frames
# We use the pandas .copy(deep=True) function to prevent the SettingWithCopyWarning we would otherwise get. Since we do
# not write, but only read from the data, the warning does not affect the data frames
df_emotion = data[['Anger', 'Boredom', 'Disgust', 'Fear', 'Happiness', 'Emo_Neutral', 'Sadness', 'Filename']].copy(deep=True)
df_affect = data[['Aggressiv', 'Cheerful', 'Intoxicated', 'Nervous', 'Aff_Neutral', 'Tired', 'Filename']].copy(deep=True)
df_loi = data[['Disinterest', 'Normal', 'High Interest', 'Filename']].copy(deep=True)
df_ar_val = data[['Arousal', 'Valence', 'Filename']].copy(deep=True)
#For further usage, we want to append the CharacterID as a column, which is saved with other information in the filename
#Since we only want the digits, we can remove all non-digit characters of the filename column and append the column to the df

df_emotion['Char_ID'] = df_emotion['Filename'].replace('\D+','', regex = True).copy(deep=True)
df_affect['Char_ID'] = df_affect['Filename'].replace('\D+','', regex = True).copy(deep=True)
df_loi['Char_ID'] = df_loi['Filename'].replace('\D+','', regex = True).copy(deep=True)
df_ar_val['Char_ID'] = df_ar_val['Filename'].replace('\D+','', regex = True).copy(deep=True)

#Load data about speakers
char_data = pd.read_csv("CHI_2019_CharacterData.csv")  
char_data.replace({'Male': 0.0, 'Female': 1.0}, inplace = True)

#Join above tables and Character Tables
#To Join DataFrames we have to cast the column on which we want to join to int, so that both columns have the same data type
char_data['ID'] = char_data['ID'].astype(int)
df_ar_val['Char_ID'] = df_ar_val['Char_ID'].astype(int)
df_emotion['Char_ID'] = df_emotion['Char_ID'].astype(int)
df_affect['Char_ID'] = df_affect['Char_ID'].astype(int)
df_loi['Char_ID'] = df_loi['Char_ID'].astype(int)

#Save new data frames
df_ar_val_char = df_ar_val.merge(char_data, how = 'left', left_on='Char_ID', right_on='ID')
df_emotion_char = df_emotion.merge(char_data, how = 'left', left_on='Char_ID', right_on= 'ID')
df_affect_char = df_affect.merge(char_data, how = 'left', left_on='Char_ID', right_on= 'ID')
df_loi_char = df_loi.merge(char_data, how = 'left', left_on='Char_ID', right_on= 'ID')

#Now, we only want to have data containing information about the answers
#For that we need to extract from the filename column, whether the file was part of an answer
#a = answer, p = presentation, q = question
#sentence_type should be the same for all tables, but just to be sure
arval_sentence_type = df_ar_val_char.Filename.str.replace('\d+','').str[3:-4]
df_ar_val_char['SentenceType'] = arval_sentence_type
emo_sentence_type = df_emotion_char.Filename.str.replace('\d+','').str[3:-4]
df_emotion_char['SentenceType'] = emo_sentence_type
aff_sentence_type = df_affect_char.Filename.str.replace('\d+','').str[3:-4]
df_affect_char['SentenceType'] = aff_sentence_type
loi_sentence_type = df_loi_char.Filename.str.replace('\d+','').str[3:-4]
df_loi_char['SentenceType'] = loi_sentence_type

#For affect, we will have to drop the intoxication column and thus we will re-normalize the other values
affect_label.remove('Intoxicated')
df_affect_char = df_affect_char.drop(['Intoxicated'], axis = 1)
norm_test = pp.normalize(df_affect_char[affect_label], norm = 'l1')
df_affect_char[affect_label] = norm_test

df_loi_char['Normal Interest'] = df_loi_char['Disinterest'] + df_loi_char['Normal']
df_loi_char = df_loi_char.drop(['Disinterest', 'Normal'], axis = 1)
loi_label = ['Normal Interest', 'High Interest']

C:\Users\user\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:49: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


In [2]:
#Now select only those who have SentenceType == 'a'
df_ar_val_char_a = df_ar_val_char.loc[df_ar_val_char['SentenceType'] == 'a']
df_emotion_char_a = df_emotion_char.loc[df_emotion_char['SentenceType'] == 'a']
df_affect_char_a = df_affect_char.loc[df_affect_char['SentenceType'] == 'a']
df_loi_char_a = df_loi_char.loc[df_loi_char['SentenceType'] == 'a']

#Now select only those who have SentenceType == 'a'
df_ar_val_char_p = df_ar_val_char.loc[df_ar_val_char['SentenceType'] == 'p'].copy()
df_emotion_char_p = df_emotion_char.loc[df_emotion_char['SentenceType'] == 'p'].copy()
df_affect_char_p = df_affect_char.loc[df_affect_char['SentenceType'] == 'p'].copy()
df_loi_char_p = df_loi_char.loc[df_loi_char['SentenceType'] == 'p'].copy()

#Now select only those who have SentenceType == 'a'
df_ar_val_char_q = df_ar_val_char.loc[df_ar_val_char['SentenceType'] == 'q']
df_emotion_char_q = df_emotion_char.loc[df_emotion_char['SentenceType'] == 'q']
df_affect_char_q = df_affect_char.loc[df_affect_char['SentenceType'] == 'q']
df_loi_char_q = df_loi_char.loc[df_loi_char['SentenceType'] == 'q']

In [3]:
IDs = df_emotion_char_p['Char_ID'].copy()
IDs.drop_duplicates()

# For presentations we need an aggregate, e.g. median for the presenters
for i in IDs:
    #first for emotion
    for l in emotion_label:    
        emo = df_emotion_char_p.loc[df_emotion_char_p.Char_ID == i,l]
        emo_neu = hp.constructMedianSeries(emo)
        df_emotion_char_p.loc[df_emotion_char_p.Char_ID == i,l] = emo_neu
    #Then for affect
    for l in affect_label:    
        aff = df_affect_char_p.loc[df_affect_char_p.Char_ID == i,l]
        aff_neu = hp.constructMedianSeries(aff)
        df_affect_char_p.loc[df_affect_char_p.Char_ID == i,l] = aff_neu  
    for l in loi_label:    
        loi = df_loi_char_p.loc[df_loi_char_p.Char_ID == i,l]
        loi_neu = hp.constructMedianSeries(loi)
        df_loi_char_p.loc[df_loi_char_p.Char_ID == i,l] = loi_neu  
    for l in ['Arousal', 'Valence']:    
        arval = df_ar_val_char_p.loc[df_ar_val_char_p.Char_ID == i,l]
        arval_neu = hp.constructMedianSeries(arval)
        df_ar_val_char_p.loc[df_ar_val_char_p.Char_ID == i,l] = arval_neu 
        
#Now let's drop the duplicate values, since we only need one row now per person
df_emotion_char_p.drop_duplicates(subset=['ID'], inplace = True)
df_affect_char_p.drop_duplicates(subset=['ID'], inplace = True)
df_loi_char_p.drop_duplicates(subset=['ID'], inplace = True)
df_ar_val_char_p.drop_duplicates(subset=['ID'], inplace = True) 

In [4]:
emotion_label.extend(['Sex', 'Academic Status', 'IsNativeSpeaker'])
affect_label.extend(['Sex', 'Academic Status', 'IsNativeSpeaker'])
loi_label.extend(['Sex', 'Academic Status', 'IsNativeSpeaker'])

df_emotion_char_p.columns = df_emotion_char_p.columns.map(lambda x: 'P_' + x if x in emotion_label else x)
df_affect_char_p.columns = df_affect_char_p.columns.map(lambda x: 'P_' + x if x in affect_label else x)
df_loi_char_p.columns = df_loi_char_p.columns.map(lambda x: 'P_' + x if x in loi_label else x)
df_ar_val_char_p.columns = df_ar_val_char_p.columns.map(lambda x: 'P_' + x if x in ['Arousal', 'Valence', 'Sex', 'Academic Status', 'IsNativeSpeaker'] else x)

df_emotion_char_a.columns = df_emotion_char_a.columns.map(lambda x: 'A_' + x if x in emotion_label else x)
df_affect_char_a.columns = df_affect_char_a.columns.map(lambda x: 'A_' + x if x in affect_label else x)
df_loi_char_a.columns = df_loi_char_a.columns.map(lambda x: 'A_' + x if x in loi_label else x)
df_ar_val_char_a.columns = df_ar_val_char_a.columns.map(lambda x: 'A_' + x if x in ['Arousal', 'Valence', 'Sex', 'Academic Status', 'IsNativeSpeaker'] else x)

df_emotion_char_q.columns = df_emotion_char_q.columns.map(lambda x: 'Q_' + x if x in emotion_label else x)
df_affect_char_q.columns = df_affect_char_q.columns.map(lambda x: 'Q_' + x if x in affect_label else x)
df_loi_char_q.columns = df_loi_char_q.columns.map(lambda x: 'Q_' + x if x in loi_label else x)
df_ar_val_char_q.columns = df_ar_val_char_q.columns.map(lambda x: 'Q_' + x if x in ['Arousal', 'Valence', 'Sex', 'Academic Status', 'IsNativeSpeaker'] else x)

df_emotion_char_p = df_emotion_char_p.drop(['Name', 'Char_ID', 'VideoTitle', 'SentenceType', 'Filename'], axis = 1)
df_affect_char_p = df_affect_char_p.drop(['Name', 'Char_ID', 'VideoTitle', 'SentenceType', 'Filename'], axis = 1)
df_loi_char_p = df_loi_char_p.drop(['Name', 'Char_ID', 'VideoTitle', 'SentenceType', 'Filename'], axis = 1)
df_ar_val_char_p = df_ar_val_char_p.drop(['Name', 'Char_ID', 'VideoTitle', 'SentenceType', 'Filename'], axis = 1)

df_emotion_char_a = df_emotion_char_a.drop(['Name', 'Char_ID', 'VideoTitle', 'SentenceType', 'Filename'], axis = 1)
df_affect_char_a = df_affect_char_a.drop(['Name', 'Char_ID', 'VideoTitle', 'SentenceType', 'Filename'], axis = 1)
df_loi_char_a = df_loi_char_a.drop(['Name', 'Char_ID', 'VideoTitle', 'SentenceType', 'Filename'], axis = 1)
df_ar_val_char_a = df_ar_val_char_a.drop(['Name', 'Char_ID', 'VideoTitle', 'SentenceType', 'Filename'], axis = 1)

df_emotion_char_q = df_emotion_char_q.drop(['Name', 'Char_ID', 'VideoTitle', 'SentenceType', 'Filename'], axis = 1)
df_affect_char_q = df_affect_char_q.drop(['Name', 'Char_ID', 'VideoTitle', 'SentenceType', 'Filename'], axis = 1)
df_loi_char_q = df_loi_char_q.drop(['Name', 'Char_ID', 'VideoTitle', 'SentenceType', 'Filename'], axis = 1)
df_ar_val_char_q = df_ar_val_char_q.drop(['Name', 'Char_ID', 'VideoTitle', 'SentenceType', 'Filename'], axis = 1)

#To do: Merge Data Frames; First Question and Presentation, then answer and question
ar_val_char = df_ar_val_char_q.merge(df_ar_val_char_p, how = 'left', left_on='VideoID', right_on='VideoID')
loi_char = df_loi_char_q.merge(df_loi_char_p, how = 'left', left_on='VideoID', right_on='VideoID')
affect_char = df_affect_char_q.merge(df_affect_char_p, how = 'left', left_on='VideoID', right_on='VideoID')
emotion_char = df_emotion_char_q.merge(df_emotion_char_p, how = 'left', left_on='VideoID', right_on='VideoID')

df_ar_val_char_a.reset_index(drop = True, inplace = True)
df_ar_val_char_q.reset_index(drop = True, inplace = True)
arval = pd.concat([ar_val_char, df_ar_val_char_a], axis = 1)
arval.drop(['ID_x', 'ID_y', 'VideoID', 'ID', 'Q_Academic Status','Q_IsNativeSpeaker', 'A_IsNativeSpeaker', 'A_Academic Status', 'P_Academic Status', 'P_IsNativeSpeaker', 'P_Sex'], axis = 1, inplace = True)

df_loi_char_a.reset_index(drop = True, inplace = True)
df_loi_char_q.reset_index(drop = True, inplace = True)
loi = pd.concat([loi_char, df_loi_char_a], axis = 1)
loi.drop(['ID_x', 'ID_y', 'VideoID', 'ID', 'Q_Academic Status','Q_IsNativeSpeaker', 'A_IsNativeSpeaker', 'A_Academic Status', 'P_Academic Status', 'P_IsNativeSpeaker', 'P_Sex'], axis = 1, inplace = True)

df_affect_char_a.reset_index(drop = True, inplace = True)
df_affect_char_q.reset_index(drop = True, inplace = True)
affect = pd.concat([affect_char, df_affect_char_a], axis = 1)
affect.drop(['ID_x', 'ID_y', 'VideoID', 'ID', 'Q_Academic Status','Q_IsNativeSpeaker', 'A_IsNativeSpeaker', 'A_Academic Status', 'P_Academic Status', 'P_IsNativeSpeaker', 'P_Sex'], axis = 1, inplace = True)

df_emotion_char_a.reset_index(drop = True, inplace = True)
df_emotion_char_q.reset_index(drop = True, inplace = True)
emotion = pd.concat([emotion_char, df_emotion_char_a], axis = 1)
emotion.drop(['ID_x', 'ID_y', 'VideoID', 'ID', 'Q_Academic Status','Q_IsNativeSpeaker', 'A_IsNativeSpeaker', 'A_Academic Status', 'P_Academic Status', 'P_IsNativeSpeaker', 'P_Sex'], axis = 1, inplace = True)
#Now we have our dataframe ready for logsitic regression:
#Each row resembles an interaction between one question, the corresponding answer and the median of the presenter
#We could do this since both dataframes are default sorted by filenames (emotion_char and df_emotion_char_a)
#So each question is in the same line as the corresponding answer
emotion

,Q_Anger,Q_Boredom,Q_Disgust,Q_Fear,Q_Happiness,Q_Emo_Neutral,Q_Sadness,Q_Sex,P_Anger,P_Boredom,...,P_Emo_Neutral,P_Sadness,A_Anger,A_Boredom,A_Disgust,A_Fear,A_Happiness,A_Emo_Neutral,A_Sadness,A_Sex
0,0.002253,0.005161,0.028762,0.002877,0.001428,0.000320,0.959199,0.0,0.005758,0.026247,...,0.003056,0.550885,0.002512,0.001862,0.144437,0.001235,0.001340,0.000104,0.848510,1.0
1,0.007885,0.004149,0.152280,0.004555,0.003764,0.000246,0.827121,1.0,0.005758,0.026247,...,0.003056,0.550885,0.003424,0.006929,0.163217,0.001225,0.001768,0.000377,0.823060,1.0
2,0.003784,0.006456,0.014762,0.002672,0.001569,0.000624,0.970133,0.0,0.005758,0.026247,...,0.003056,0.550885,0.000404,0.000340,0.071035,0.000072,0.000371,0.000017,0.927761,1.0
3,0.004143,0.000882,0.151633,0.000632,0.002108,0.000065,0.840536,0.0,0.005758,0.026247,...,0.003056,0.550885,0.009743,0.036144,0.171343,0.004482,0.005685,0.003127,0.769477,1.0
4,0.008246,0.043610,0.119756,0.005309,0.007007,0.012636,0.803436,0.0,0.005346,0.016413,...,0.002316,0.921682,0.004123,0.015522,0.088884,0.002106,0.001884,0.001089,0.886391,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,0.001621,0.052537,0.003183,0.011076,0.002419,0.003676,0.925487,0.0,0.051643,0.210241,...,0.028904,0.413051,0.001201,0.012959,0.096147,0.001087,0.001077,0.000864,0.886665,0.0
191,0.002563,0.004989,0.018547,0.005339,0.001827,0.000377,0.966359,0.0,0.051643,0.210241,...,0.028904,0.413051,0.002432,0.013194,0.072456,0.001132,0.002074,0.000934,0.907778,0.0
192,0.001638,0.009019,0.005033,0.020095,0.002618,0.000631,0.960965,0.0,0.051643,0.210241,...,0.028904,0.413051,0.001550,0.012946,0.088773,0.001038,0.001073,0.000799,0.893822,0.0
193,0.000195,0.004939,0.001564,0.000372,0.000364,0.000408,0.992159,0.0,0.051643,0.210241,...,0.028904,0.413051,0.010152,0.039024,0.194922,0.003860,0.006281,0.004162,0.741597,0.0
